In [50]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import spacy
import re
import warnings
import copy

warnings.filterwarnings('ignore', message='Discarded redundant search for Synset')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt') 
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('omw-1.4')
nlp = spacy.load("en_core_web_sm")

from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
Word2Vec_corpus = api.load('text8') 
Word2Vec_model = Word2Vec(Word2Vec_corpus) 
glove_model = api.load("glove-twitter-25") 

from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from nltk.chunk import ne_chunk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.utils import deaccent
from sklearn.decomposition import LatentDirichletAllocation as LDA
from spacy.matcher import Matcher

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/leon/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/leon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/leon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/leon/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/leon/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/leon/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/leon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloadin

In [51]:
files = ['Data/Customer_review_data/Apex AD2600 Progressive-scan DVD player.txt',
         'Data/Customer_review_data/Canon G3.txt',
         'Data/Customer_review_data/Creative Labs Nomad Jukebox Zen Xtra 40GB.txt',
         'Data/Customer_review_data/Nikon coolpix 4300.txt',
         'Data/Customer_review_data/Nokia 6610.txt',
         'Data/CustomerReviews-3_domains/Computer.txt',
         'Data/CustomerReviews-3_domains/Router.txt',
         'Data/CustomerReviews-3_domains/Speaker.txt',
         'Data/Reviews-9-products/Canon PowerShot SD500.txt',
         'Data/Reviews-9-products/Canon S100.txt',
         'Data/Reviews-9-products/Diaper Champ.txt',
         'Data/Reviews-9-products/Hitachi router.txt',
         'Data/Reviews-9-products/ipod.txt',
         'Data/Reviews-9-products/Linksys Router.txt',
         'Data/Reviews-9-products/MicroMP3.txt',
         'Data/Reviews-9-products/Nokia 6600.txt',
         'Data/Reviews-9-products/norton.txt']

In [52]:
def read_file(file_path):
    tagged_reviews = []
    
    with open(file_path, 'r') as file:
        text = file.read()
        reviews = text.strip().split('\n')

        if reviews[0] == '*' * 77:
            reviews = reviews[11:]

        reviews = pre_process_review(reviews)
        
        for review in reviews:
            parts = review.split('##')
            
            if len(parts) > 1:
                tags = parts[0].strip().split(',')
                content = parts[1].strip() 
            else:
                tags = []
                content = parts
                
            tagged_reviews.append({'Tags': tags, 'Review': content})

        df = pd.DataFrame(tagged_reviews)
        df.attrs['title'] = file_path.split('/')[-1]

        return df

In [53]:
# Pre-processing functions


def pre_process_review(reviews):

    processed_reviews = []
    title_switch = False
    title = ''

    for review in reviews:
        if review.startswith('[t]'):
            title = review[3:]
            title_switch = True
        elif title_switch:
            appended_review = review + title 
            processed_reviews.append(appended_review)
            title_switch = False
            title = ''
        else:
            processed_reviews.append(review)

    return processed_reviews



def preserve_compound_phrases(text):
    doc = nlp(text)
    processed_tokens = []
    
    for token in doc:
        if token.dep_ in ('compound', 'amod') and token.head.pos_ == 'NOUN':
            compound_phrase = token.text + "_" + token.head.text
            if compound_phrase not in processed_tokens:
                processed_tokens.append(compound_phrase)
        elif token.pos_ == 'NOUN' and any(child.dep_ == 'compound' for child in token.children):
            continue
        else:
            processed_tokens.append(token.text)
    
    return processed_tokens
    



def chuncking_post_process(text):
    words = text.split()
    processed_words = []
    
    i = 0
    while i < len(words):
        current_word = words[i]
        processed_words.append(current_word)
        
        if '_' in current_word:
            i += 1
            while i < len(words) and '_' in words[i]:
                processed_words.append(words[i])
                i += 1
            if i < len(words):
                i += 1
        else:
            i += 1  
            
    return ' '.join(processed_words)


In [54]:
def pre_processing_controller(df):
    
    df['Tokenised_Review'] = df['Review'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x) 
    df['Tokenised_Review'] = df['Tokenised_Review'].apply(lambda review: preserve_compound_phrases(review))
    df['Soft_Filtered_Review'] = df['Tokenised_Review'].apply(lambda tokens: [token.lower() for token in tokens if ("_" in token) or (token.isalpha() and token.lower())])
    df['Soft_Filtered_Review_String'] = df['Soft_Filtered_Review'].apply(lambda tokens: ' '.join(tokens))
    df['Soft_Filtered_Review_String'] = df['Soft_Filtered_Review_String'].apply(chuncking_post_process)
    df['Filtered_Review'] = df['Tokenised_Review'].apply(lambda tokens: [token.lower() for token in tokens if ("_" in token) or (token.isalpha() and token.lower() not in stop_words)])
    df['Filtered_Review_String'] = df['Filtered_Review'].apply(lambda tokens: ' '.join(tokens))
    df['Lemmatised_Review_String'] = df['Filtered_Review_String'].apply(lambda review_string: " ".join([token.lemma_ for token in nlp(review_string)]))
    df['Lemmatised_Tokenised_Filtered_Review'] = df['Filtered_Review_String'].apply(lambda review: word_tokenize(review))
    df['Stemmed_Review'] = df['Lemmatised_Tokenised_Filtered_Review'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    df['Stemmed_Review_String'] = df['Stemmed_Review'].apply(lambda tokens: ' '.join(tokens))
    
    return df 

In [55]:
def display_freq_dist(df):
    all_words = [word for review in df['Filtered_Review'] for word in review]
    freq_dist = FreqDist(all_words)
    top_items = sorted(freq_dist.items(), key=lambda x: x[1], reverse=True)[:20]
    words, frequencies = zip(*top_items)
    plt.figure(figsize=(6, 3))  
    plt.bar(words, frequencies, color='skyblue')  
    plt.xlabel('Words') 
    plt.ylabel('Frequency') 
    plt.title('Top Words Frequency Distribution')  
    plt.xticks(rotation=45) 
    plt.show()

# display_freq_dist(df)

In [56]:
def k_means(string_list):

    num_clusters = 2

    tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_df=0.85, min_df=2)
    tfidf_matrix = tfidf_vectorizer.fit_transform(string_list)
    
    km = KMeans(n_clusters=num_clusters, n_init=10)
    km.fit(tfidf_matrix)
    clusters = km.labels_.tolist()
    
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]
    terms = tfidf_vectorizer.get_feature_names_out()
    
    for i in range(num_clusters):
        top_terms = [terms[ind] for ind in order_centroids[i, :10]]  # Get top 10 terms for each cluster
        print(f"Cluster {i}: {top_terms}")
    
    pca = PCA(n_components=2)
    reduced_data = pca.fit_transform(tfidf_matrix.toarray())
    
    # Get the cluster labels for each data point
    cluster_labels = km.labels_
    
    plt.figure(figsize=(8, 4))  # Set figure size
    
    # Scatter plot of the reduced data, colored by cluster labels
    plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=cluster_labels, cmap='viridis', s=50, alpha=0.6)
    
    # Adding labels for axes
    plt.xlabel('PCA 1')
    plt.ylabel('PCA 2')
    
    # Title of the plot
    plt.title('2D Visualization of K-Means Clusters')
    
    # Display the plot
    print('\n')
    plt.show()

# k_means(df['Filtered_Review_String'])
# k_means(df['Lemmatised_Review_String'])
# k_means(df['Stemmed_Review_String'])

In [57]:
def POS_Noun_Tagging(string_list):
    
    reviews = string_list.tolist()
    features = []
    
    for review in reviews:
        tokens = word_tokenize(review)
        tagged = pos_tag(tokens)
        # Extracts nouns from POS tagged text as nouns likely features names
        features.extend([word.lower() for word, tag in tagged if tag in ['NN', 'NNS', 'NNP', 'NNPS']])
    
    feature_counts = Counter(features)
    common_features = feature_counts.most_common(15)
    # display(common_features)

    return common_features

In [58]:
def Noun_Phrase_Chuncking(string_list):
    
    all_noun_phrases = []
    
    for review in string_list:
        doc = nlp(review)
        noun_phrases = [chunk.text.lower() for chunk in doc.noun_chunks]
        all_noun_phrases.extend(noun_phrases)
    
    # Count the occurrences of each noun phrase
    from collections import Counter
    phrase_counts = Counter(all_noun_phrases).most_common(15)
    
    # Display most common noun phrases
    common_phrases = phrase_counts
    # display(common_phrases)
    
    return common_phrases

In [59]:
# --------------------- Concrete noun filter 1 ---------------------

def is_concrete_noun(word):
    """Enhanced check if a noun is concrete based on its hypernyms and usage context."""
    concrete_clues = {'object', 'artifact', 'instrumentality', 'container', 'device'}
    synsets = wn.synsets(word, pos=wn.NOUN)
    for synset in synsets:
        for hyper in synset.closure(lambda s: s.hypernyms()):
            if concrete_clues.intersection(set(hyper.lemma_names())):
                return True
    return False


def context_based_filter(nouns):
    """Filter nouns based on enhanced concrete checks and contextual usage."""
    filtered_features = []
    for noun in nouns:
        if is_concrete_noun(noun):
            filtered_features.append(noun)
    return filtered_features




# --------------------- Concrete noun filter 2 ---------------------

# Define a list of concrete domains (as WordNet synsets)
concrete_domains = [
    wn.synset('artifact.n.01'),  # Artifacts, objects made by humans
    wn.synset('device.n.01'),    # Devices, tools or instruments
    wn.synset('instrumentality.n.03'),  # Instrumentalities, means of achieving an end
    # Add more domains as necessary
]

def is_related_to_concrete_domain(word):
    """
    Check if a word is semantically related to predefined concrete domains.
    """
    synsets = wn.synsets(word, pos=wn.NOUN)
    for synset in synsets:
        for domain in concrete_domains:
            if domain in synset.closure(lambda s: s.hypernyms()):
                return True
    return False


def filter_nouns_semantically(nouns):
    """
    Filter a list of nouns, keeping only those related to concrete domains.
    """
    return [noun for noun in nouns if is_related_to_concrete_domain(noun)]

In [60]:
def similarity_filter(word_tuple_list):
    
    # Define a threshold for filtering; this is arbitrary and might need adjustment
    similarity_threshold = 0.2
    
    # Extract just the words for similarity comparison
    words = [word for word, _ in word_tuple_list]
    
    # Calculate average similarities and filter
    filtered_words = []
    for word in words:
        try:
            # Calculate similarity using Word2Vec
            w2v_similarity = Word2Vec_model.wv.similarity(words[0], word)
            # Get the top N similar words using GloVe and convert to a dictionary for easier access
            glove_similar_words = dict(glove_model.most_similar(words[0], topn=10))
            # Get the similarity score for the current word from the GloVe model
            glove_similarity = glove_similar_words.get(word, 0)  # Default to 0 if word is not found
            # Average the similarities
            avg_similarity = (w2v_similarity + glove_similarity) / 2
            # Filter based on the threshold
            if avg_similarity >= similarity_threshold:
                # print(word, w2v_similarity, glove_similarity)
                filtered_words.append(word)
        except KeyError:
            # Word not in vocabulary
            # print(f"Word '{word}' not found in one of the model's vocabulary.")
            pass

    return filtered_words

In [61]:
def further_similar_filter(noun_list):
    
    main_word = 'picture'
    word = 'pic'
    
    w2v_similarity = Word2Vec_model.wv.similarity(main_word, word)
    glove_similar_words = dict(glove_model.most_similar(main_word, topn=10))
    glove_similarity = glove_similar_words.get(word, 0) 
    
    items_to_remove = []
    for word in noun_list:
        glove_similar_words = dict(glove_model.most_similar(main_word, topn=10))
        glove_similarity = glove_similar_words.get(word, 0) 
        if glove_similarity > 0:
            items_to_remove.append(word)
    
    filtered_list = [item for item in noun_list if item not in items_to_remove]
    # print(filtered_list)
    return filtered_list

In [62]:
def create_product_features_dict(product_info):
    # Extract the product title
    product_title = product_info[0]

    # Extract the features
    product_features = product_info[1:]

    # Create the dictionary with the desired structure
    product_dict = {
        product_title: {
            feature: {"positive": 0, "negative": 0} for feature in product_features
        }
    }

    return product_dict

In [63]:
def build_featured_df(df, feature_list_pos_noun):
    
    feature_df = pd.DataFrame()
    feature_list = feature_list_pos_noun[1:]
    
    featured_items_list = []
    
    for idx, review in df.iterrows():
        tokenised_review = review['Filtered_Review']
        # Find the features present in the tokenised_review
        featured_items = [item for item in feature_list if item in tokenised_review]
            
        if featured_items:
            # Convert the review Series to a DataFrame with one row
            review_df = review.to_frame().transpose()
            
            # New: Append the found features as a string (or you can keep it as list)
            review_df['Featured_Items'] = [', '.join(featured_items)]  # As a single string of items
    
            review_df['Main_Index'] = idx
            /
            # Append this review to the feature_df
            feature_df = pd.concat([review_df, feature_df], axis=0)
            
            # Additionally, append the found features to the featured_items_list
            featured_items_list.append(featured_items)

    return feature_df

In [64]:
def sentiment_classifier_1(feature_df):
    sentiment_list = []
    
    for idx, entry in feature_df.iterrows():
        review = entry['Soft_Filtered_Review_String']
        
        if len(entry['Featured_Items'].split(',')) == 1:
            
            pos_score_1, neg_score_1 = get_phrase_sentiment_1(review)
            pos_score_2, neg_score_2 = get_phrase_sentiment_2(review)
            avg_pos_score = (pos_score_1 + pos_score_2) / 2
            avg_neg_score = (neg_score_1 + neg_score_2) / 2
        
            if avg_pos_score > avg_neg_score:
                sentiment = 1
            else:
                sentiment = -1 

            sentiment_list.append(sentiment)
        else:
            sentiment_list.append(0)

    feature_df['Sentiment'] = sentiment_list
    return feature_df


In [65]:
def feature_dict_count(feature_df, product_dict):
    
    sentiment_class_list = feature_df[['Featured_Items', 'Sentiment']]
    
    for idx, row in sentiment_class_list.iterrows():
        feature = row['Featured_Items']
        sentiment = row['Sentiment']
    
        if sentiment == 1:
            val = product_dict[list(product_dict.keys())[0]][feature]['positive'] + 1
            product_dict[list(product_dict.keys())[0]][feature]['positive'] = val
        elif sentiment == -1:
            val = product_dict[list(product_dict.keys())[0]][feature]['negative'] + 1
            product_dict[list(product_dict.keys())[0]][feature]['negative'] = val
        else:
            # print('More than two features, cannot classify')
            pass

    return product_dict
    

# feature_dict = feature_dict_count(feature_df, product_dict)
# display(feature_dict)

In [88]:
def df_filter(df):
    df = df[['Main_Index', 'Featured_Items', 'Sentiment', 'Tags', 'Review', 'Tokenised_Review', 'Soft_Filtered_Review', 'Soft_Filtered_Review_String', 'Filtered_Review', 'Filtered_Review_String', 'Lemmatised_Review_String', 'Lemmatised_Tokenised_Filtered_Review', 'Stemmed_Review', 'Stemmed_Review_String']]
    df = df.set_index('Main_Index', drop=True) 
    df.index.name = 'Main Index'
    return df

In [89]:
# ********************************************************************************************************************************************

In [90]:
def LDA_Model(tokenised_reviews):
    
    # Create a dictionary representation of the documents
    dictionary = corpora.Dictionary(tokenised_reviews)
    
    # Convert dictionary to a bag of words corpus
    corpus = [dictionary.doc2bow(text) for text in tokenised_reviews]
    
    # Number of topics
    num_topics = 5
    
    # Generate LDA model
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=100, update_every=1, passes=10, alpha='auto')
    
    # Print the topics
    topics = lda_model.print_topics(num_words=5)
    for topic in topics:
        print(topic)
        

# LDA_Model(df['Stemmed_Review'])
# print('\n')
# LDA_Model(df['Filtered_Review'])
# print('\n')
# LDA_Model(df['Lemmatised_Tokenised_Filtered_Review'])

In [91]:
def LDA_Model_2(string_reviews):
    
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(string_reviews)
    
    num_topics = 5
    lda = LDA(n_components=num_topics)
    lda.fit_transform(tfidf_matrix)
    
    # Explore the topics
    terms = tfidf_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(lda.components_):
        print(f"Topic #{topic_idx+1}:")
        print(" ".join([terms[i] for i in topic.argsort()[:-10 - 1:-1]]))
        print('\n')

# LDA_Model_2(df['Lemmatised_Review_String'])

In [92]:
"""
Getting the sentiment of multiword phrases using SentiWordNet (SWN) through the NLTK library requires a more nuanced approach compared to single words because SentiWordNet does not directly provide sentiment scores for phrases. Instead, you can average the sentiment scores of individual words in the phrase, adjust the methodology to consider the phrase's context, or use compound term lookup techniques where applicable. Here’s an approach to approximate sentiment for multiword phrases:

### Step 1: Handle Compound Terms

Some compound terms might be recognized by SentiWordNet if connected by underscores (e.g., "well_known"). Before splitting the phrase into individual words, check if the compound term has an entry in SentiWordNet.

### Step 2: Average Sentiment Scores of Individual Words

For phrases not recognized as compound terms, calculate the average sentiment scores of the individual words. Consideration of part-of-speech tags can enhance accuracy, but for simplicity, this example ignores POS tags.

### Note:

- This method assumes equal weighting of words in calculating the average sentiment, which might not reflect the actual sentiment conveyed by the phrase.
- Handling negations and intensifiers (e.g., "not" in "not good", "very" in "very good") requires more sophisticated logic, as they can significantly alter the sentiment.
- Advanced models designed for sentiment analysis at the sentence or document level (like BERT-based models) may provide more accurate sentiment assessments for phrases and sentences by considering the broader context.

This approach gives a basic approximation but is limited by the nuances of natural language. For more accurate sentiment analysis on phrases or sentences, consider using pre-trained sentiment analysis models or services.
"""


def get_word_sentiment(word):
    synsets = list(swn.senti_synsets(word))
    if synsets:
        return synsets[0].pos_score(), synsets[0].neg_score()
    else:
        return 0, 0


def get_phrase_sentiment_1(phrase):
    # Try the phrase directly (useful for compound terms recognized by SWN)
    pos_score, neg_score = get_word_sentiment(phrase.replace(" ", "_"))
    if pos_score or neg_score:
        return pos_score, neg_score
    
    # Split the phrase into individual words and average their sentiment scores
    words = phrase.split()
    total_pos, total_neg = 0, 0
    for word in words:
        pos, neg = get_word_sentiment(word)
        total_pos += pos
        total_neg += neg
    avg_pos = total_pos / len(words) if words else 0
    avg_neg = total_neg / len(words) if words else 0
    
    return avg_pos, avg_neg



"""
This version attempts to address negations directly before a word and could be extended to consider intensifiers (like "very") by further
modifying the sentiment scores. For even more nuanced sentiment analysis, exploring deep learning models trained specifically for sentiment 
analysis is recommended.
"""


def penn_to_wn(tag):
    """Converts Penn Treebank tags to WordNet tags."""
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

def get_word_sentiment_2(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag:
        synsets = list(swn.senti_synsets(word, wn_tag))
        if synsets:
            return synsets[0].pos_score(), synsets[0].neg_score()
    return 0, 0

def adjust_scores_for_negation_and_intensifiers(scores, words, i):
    """Adjusts sentiment scores based on negations and intensifiers around the i-th word."""
    if i > 0 and words[i-1].lower() in ["not", "no"]:
        return -scores[0], -scores[1]  # Inverting the sentiment
    # Further adjustments for intensifiers (like "very") can be added here
    return scores

def get_phrase_sentiment_2(phrase):
    words = word_tokenize(phrase)
    tagged = pos_tag(words)
    total_pos, total_neg = 0, 0
    
    for i, (word, tag) in enumerate(tagged):
        scores = get_word_sentiment_2(word, tag)
        scores = adjust_scores_for_negation_and_intensifiers(scores, words, i)
        total_pos += scores[0]
        total_neg += scores[1]
    
    avg_pos = total_pos / len(words) if words else 0
    avg_neg = total_neg / len(words) if words else 0
    
    return avg_pos, avg_neg

In [93]:
def disambiguate_word_sense(sentence, word):
    # Use Lesk algorithm for WSD
    sense = lesk(nltk.word_tokenize(sentence), word)
    if not sense:
        return None
    
    # Get sentiment scores
    senti_synset = swn.senti_synset(sense.name())
    return {
        'word': word,
        'synset_name': sense.name(),
        'definition': sense.definition(),
        'examples': sense.examples(),
        'positivity_score': senti_synset.pos_score(),
        'negativity_score': senti_synset.neg_score(),
        'objectivity_score': senti_synset.obj_score()
    }


# review = df.iloc[200]
# for word in review['Filtered_Review']:
#     disambiguated_sense = disambiguate_word_sense(review['Review'], word)
    # print(disambiguated_sense)
    # print('\n')

In [94]:
def preprocess_and_ner(tokens):
    
    tagged = pos_tag(tokens)
    named_entities = ne_chunk(tagged)
    return named_entities

# tokenised_text = df['Filtered_Review'].iloc[106]
# named_entities = preprocess_and_ner(tokenised_text)
# print(named_entities)

In [95]:
def tf_idf(reviews):
    # Initialize the TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    
    # Transform the reviews into a TF-IDF matrix
    tfidf_matrix = tfidf_vectorizer.fit_transform(reviews)
    
    # Extract the feature names/terms from the TF-IDF Vectorizer
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    # Calculate the average TF-IDF score for each term across all documents
    scores = tfidf_matrix.mean(axis=0)
    term_scores = {feature_names[col]: scores[0, col] for col in range(scores.shape[1])}
    
    # Sort the terms by their average TF-IDF score in descending order
    sorted_term_scores = sorted(term_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Optionally: Display the top 10 terms with the highest average TF-IDF scores
    print("Top 15 terms by average TF-IDF score:")
    for term, score in sorted_term_scores[:15]:
        print(f"Term: {term}, Score: {round(score, 4)}")
    
    # Calculate cosine similarity among the documents using the TF-IDF matrix
    cos_sim_matrix = cosine_similarity(tfidf_matrix)
    
    # return tfidf_matrix, feature_names, cos_sim_matrix

# tf_idf(df['Filtered_Review_String'])

In [96]:
matcher = Matcher(nlp.vocab)

# Define custom patterns
patterns = [
    [{"POS": "ADJ"}, {"POS": "NOUN", "OP": "+"}],  # Adjective followed by one or more nouns
    [{"POS": "NOUN", "OP": "+"}, {"LOWER": "mode"}]  # One or more nouns followed by "mode"
]
matcher.add("CUSTOM_PATTERNS", patterns)


def POS_Chuck_Parser_Matcher(review):
    doc = nlp(review)
    matches = matcher(doc)

    extracted_phrases = []
    for match_id, start, end in matches:
        span = doc[start:end]
        extracted_phrases.append(span.text)

    print(extracted_phrases)

In [97]:
def POS_Chuck_Parser(review):

    doc = nlp(review)
    
    # Initialize a list to hold our extracted phrases
    extracted_phrases = []
    
    # Iterate over tokens in the doc
    for token in doc:
        # Look for an adverb modifying an adjective and check the adjective doesn't have a noun child
        if token.pos_ == "ADV" and token.head.pos_ == "ADJ":
            is_adj_modified = False
            for child in token.head.children:
                if child.dep_ in ["attr", "dobj", "pobj"]:  # The adjective is modifying a noun
                    is_adj_modified = True
                    break
            if not is_adj_modified:
                # Capture the adverb-adjective pair "rather heavy"
                extracted_phrases.append(token.text + " " + token.head.text)
    
        # Look for an adjective modifying a noun and check if it's in a prepositional phrase
        if token.pos_ == "ADJ" and token.head.pos_ in ["NOUN", "PROPN"]:
            is_in_prep_phrase = False
            for ancestor in token.head.ancestors:
                if ancestor.dep_ == "prep":
                    is_in_prep_phrase = True
                    breakg
            if not is_in_prep_phrase:
                # Capture the adjective-noun pair "great camera"
                extracted_phrases.append(token.text + " " + token.head.text)

    print(extracted_phrases)

# index = 24

# print(df['Tags'].iloc[index])
# print(df['Review'].iloc[index])

# POS_Chuck_Parser(df['Review'].iloc[index])
# POS_Chuck_Parser(df['Filtered_Review_String'].iloc[index])
# POS_Chuck_Parser(df['Lemmatised_Review_String'].iloc[index])

# POS_Chuck_Parser_Matcher(df['Review'].iloc[index])
# POS_Chuck_Parser_Matcher(df['Filtered_Review_String'].iloc[index])
# POS_Chuck_Parser_Matcher(df['Lemmatised_Review_String'].iloc[index])

In [98]:
def opinion_miner_controller(file):
    
    df = read_file(file)
    df = pre_processing_controller(df)
    nouns = POS_Noun_Tagging(df['Lemmatised_Review_String'])
    similar_nouns = similarity_filter(nouns)
    features = further_similar_filter(similar_nouns)

    if len(features) == 1:
        return print(f'Only item name extracted: {features[0]} - No other features')
    else:
        feature_df = build_featured_df(df, features)
        feature_df = sentiment_classifier_1(feature_df)
        feature_df = df_filter(feature_df)
        product_dict = create_product_features_dict(features)
        feature_dict = feature_dict_count(feature_df, copy.deepcopy(product_dict))
        return feature_dict, feature_df

In [99]:
# for file in files:
#     feature_dict = opinion_miner_controller(file)
#     print(feature_dict)
#     print('\n')

feature_dict, feature_df = opinion_miner_controller(files[1])
display(feature_dict)
display(feature_df)

{'camera': {'picture': {'positive': 8, 'negative': 10},
  'shoot': {'positive': 18, 'negative': 4},
  'flash': {'positive': 10, 'negative': 8},
  'photo': {'positive': 1, 'negative': 0}}}

,Featured_Items,Sentiment,Tags,Review,Tokenised_Review,Soft_Filtered_Review,Soft_Filtered_Review_String,Filtered_Review,Filtered_Review_String,Lemmatised_Review_String,Lemmatised_Tokenised_Filtered_Review,Stemmed_Review,Stemmed_Review_String
Main Index,,,,,,,,,,,,,
596,shoot,1,"[weight[-1], camera[+2]]",rather heavy for point and shoot but a great c...,"[rather, heavy, for, point, and, shoot, but, a...","[rather, heavy, for, point, and, shoot, but, a...",rather heavy for point and shoot but a great_c...,"[rather, heavy, point, shoot, great_camera, ca...",rather heavy point shoot great_camera camera s...,rather heavy point shoot great_camera camera s...,"[rather, heavy, point, shoot, great_camera, ca...","[rather, heavi, point, shoot, great_camera, ca...",rather heavi point shoot great_camera camera s...
588,flash,1,[],"the canon allows you to change lenses , and it...","[the, canon, allows, you, to, change, lenses, ...","[the, canon, allows, you, to, change, lenses, ...",the canon allows you to change lenses and it a...,"[canon, allows, change, lenses, accepts, ibm, ...",canon allows change lenses accepts ibm microdr...,canon allow change lense accept ibm microdrive...,"[canon, allows, change, lenses, accepts, ibm, ...","[canon, allow, chang, lens, accept, ibm, micro...",canon allow chang lens accept ibm microdr type...
585,picture,1,[],the canon g3 improves in almost all ways on th...,"[the, canon, g3, improves, in, almost, all, wa...","[the, canon, improves, in, almost, all, ways, ...",the canon improves in almost all ways on the a...,"[canon, improves, almost, ways, fact, beats, n...",canon improves almost ways fact beats nikon co...,canon improve almost way fact beat nikon coolp...,"[canon, improves, almost, ways, fact, beats, n...","[canon, improv, almost, way, fact, beat, nikon...",canon improv almost way fact beat nikon coolpi...
583,shoot,-1,[],"it can be a point-and-shoot camera , but if yo...","[it, can, be, a, point, -, and, -, shoot, came...","[it, can, be, a, point, and, shoot, camera, bu...",it can be a point and shoot camera but if you ...,"[point, shoot, camera, intend, get, finer_feat...",point shoot camera intend get finer_features s...,point shoot camera intend get finer_feature su...,"[point, shoot, camera, intend, get, finer_feat...","[point, shoot, camera, intend, get, finer_feat...",point shoot camera intend get finer_featur sug...
566,shoot,1,[],the bottom line : if you 're serious about you...,"[the, bottom_line, line, :, if, you, 're, seri...","[the, bottom_line, line, if, you, serious, abo...",the bottom_line if you serious about your digi...,"[bottom_line, line, serious, digital_photograp...",bottom_line line serious digital_photography p...,bottom_line line serious digital_photography p...,"[bottom_line, line, serious, digital_photograp...","[bottom_lin, line, seriou, digital_photographi...",bottom_lin line seriou digital_photographi pho...
548,flash,-1,[],this has made the greatest difference in the q...,"[this, has, made, the, greatest_difference, di...","[this, has, made, the, greatest_difference, di...",this has made the greatest_difference in the q...,"[made, greatest_difference, difference, qualit...",made greatest_difference difference quality ph...,make greatest_difference difference quality ph...,"[made, greatest_difference, difference, qualit...","[made, greatest_differ, differ, qualiti, photo...",made greatest_differ differ qualiti photograph...
536,flash,1,[],i 've been using a powershot s330 for nearly a...,"[i, ', ve, been, using, a, powershot, s330, fo...","[i, ve, been, using, a, powershot, for, nearly...",i ve been using a powershot for nearly a year ...,"[using, powershot, nearly, year, wanted, upgra...",using powershot nearly year wanted upgrade meg...,use powershot nearly year want upgrade megapix...,"[using, powershot, nearly, year, wanted, upgra...","[use, powershot, nearli, year, want, upgrad, m...",use powershot nearli year want upgrad megapixe...
534,picture,-1,[],"however 